# Парсинг данных

В рамках данного проекта необходимо осуществить парсинг данных с веб-сайта. В качества веб-сайта взят habr.com. Необходимо осуществить парсинг лучших постов за все время. Данные для парсинга: 1) параметры - наименование статьи, дата загрузки, категория, к которой пост относится; количество голосов и 2) целевой показатель - число просмотров. Параметр 'количество голосов' можно также использовать как целевой показатель, поэтому и информация по нему была собрана. Однако в качестве независимого параметра он не подходит, поскольку может сильно зависеть от количества просмотров.

В дальнейшем можно рассмотреть не только лучшие за все время посты, но и увеличить выборку еще б**о**льшим объемом повседневных постов.

Объем выборки составил 997 статей (для 3 не было обнаружено количество просмотров, поэтому от них решено было избавиться. В дополнение, результат работы парсера (база данных) был сохранен в отдельный файл 'posts_data.csv')

<br>**План проекта**:
1. Импортирование библиотек
2. Парсинг данных
3. Сохранение базы данных

## 1. Импортирование библиотек

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import re
import pandas as pd

'''
from bs4 import BeautifulSoup
import os
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import cycle
'''

'\nfrom bs4 import BeautifulSoup\nimport os\nimport concurrent.futures\nfrom concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor\nfrom itertools import cycle\n'

## 2. Парсинг данных

In [13]:
firefox_opt = Options()
firefox_opt.page_load_strategy = 'eager'
driver = webdriver.Firefox(options=firefox_opt)

habr_url = 'https://habr.com/ru/top/alltime/'

In [14]:
# Функция для получения данных с одной веб-страницы
def get_page_data(page_num):
    # Заход на веб-страницу
    page_url = f'{habr_url}page{page_num}'
    driver.get(page_url)
    driver.implicitly_wait(5)
    
    # Прокрутка до конца страницы
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Наименования постов
    titles = [title.text for title in driver.find_elements(By.CLASS_NAME, 'tm-title__link')]
    
    # Количество просмотров
    view_counts_link = driver.find_elements(By.CLASS_NAME, 'tm-icon-counter__value')
    view_counts = []
    for view_count in view_counts_link:
        if 'M' in view_count.text:
            view_counts.append(float(re.split('M', view_count.text)[0]) * 1000)
        elif 'K' in view_count.text:
            view_counts.append(float(re.split('K', view_count.text)[0]))
    
    # Категории
    categories_link = driver.find_elements(By.CLASS_NAME, 'tm-article-snippet__hubs-container')
    categories = []
    for category in categories_link:
        categories.append([i for i in re.split('\n', category.text) if i != '*'])

    # Дата и время публикации
    dates_link = driver.find_elements(By.CSS_SELECTOR, 'time')
    dates = [date.get_attribute('datetime') for date in dates_link if 'в' in date.text]
    
    # Количество голосов
    votes_link = driver.find_elements(By.CLASS_NAME, 'tm-votes-meter.tm-data-icons__item')
    votes = [vote.text.split('+')[0] for vote in votes_link]
    
    # Объединение собранных данных в датафрейм
    df = pd.concat(
        [pd.Series(titles), pd.Series(view_counts), pd.Series(categories), pd.Series(dates)],
        axis=1
    )
    df.columns = ['title', 'view_count', 'category', 'date']
    
    return df

In [15]:
# Получим данные с каждой страницы топ-50 по постам
df_posts = pd.DataFrame(columns=['title', 'view_count', 'category', 'date'])

for n in range(1, 51):
    df_posts = df_posts.append(get_page_data(n), ignore_index=True)

driver.quit()

In [23]:
# Получим количество пропущенных значений по каждому столбцу
df_posts.isna().sum()

title         0
view_count    0
category      0
date          0
dtype: int64

In [21]:
# Избавимся от постов, для которых не было обнаружено 'view_count' (их всего 3) 
df_posts = df_posts[~df_posts['view_count'].isna()].reset_index(drop=True)
df_posts.head()

,title,view_count,category,date
0,Делаем приватный монитор из старого LCD монитора,958.0,[DIY или Сделай сам],2011-11-27T19:21:13.000Z
1,Самый беззащитный — уже не Сапсан. Всё оказало...,524.0,"[Информационная безопасность, Системное админи...",2021-01-13T05:51:41.000Z
2,Были получены исходники 3300 глобальных интерн...,268.0,[Информационная безопасность],2009-09-23T09:17:27.000Z
3,История игрушки. Поле Чудес,290.0,"[История IT, Игры и игровые консоли]",2011-07-18T08:23:11.000Z
4,"[Обновлено в 10:52, 14.12.19] В офисе Nginx пр...",311.0,"[Блог компании ITSumma, Nginx, Законодательств...",2019-12-12T12:34:52.000Z


## 3. Сохранение данных

In [24]:
df_posts.to_csv('posts_data.csv')